In [2]:
# importing the module
import cv2
import pandas as pd
import numpy as np
import random as rnd
import os

# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
#jupyter nbconvert inspect.ipynb --no-input --no-prompt



In [15]:
# function to display the coordinates of
# of the points clicked on the image
def click_event(event, x, y, flags, params):
 
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
 
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
 
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', img)
 
    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
 
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
 
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        b = img[y, x, 0]
        g = img[y, x, 1]
        r = img[y, x, 2]
        cv2.putText(img, str(b) + ',' +
                    str(g) + ',' + str(r),
                    (x,y), font, 1,
                    (255, 255, 0), 2)
        cv2.imshow('image', img)
 

path= '/media/alb/Elements/DATASET/SoccerNet/tracking/test/SNMOT-150/img1/000001.jpg'

# driver function
if __name__=="__main__":
 
    # reading the image
    img = cv2.imread(path, 1)
 
    # displaying the image
    cv2.imshow('image', img)
 
    # setting mouse handler for the image
    # and calling the click_event() function
    cv2.setMouseCallback('image', click_event)
 
    # wait for a key to be pressed to exit
    cv2.waitKey(0)
 
    # close the window
    cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:967: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [ ]:
path_to_txt_files ='/media/alb/Elements/DATASET/SoccerNet/tracking/test/SNMOT-150/'
path_det = path_to_txt_files + '/det/det.txt'
path_gt = path_to_txt_files + '/gt/gt.txt'

path_to_save ='/media/alb/Elements/DATASET/SoccerNet/tracking/proj_DL/proj_test'
save_gt = path_to_save + '/proj_gt'
save_det = path_to_save + '/proj_det'
save_img1 = path_to_save + '/proj_img1'

gt = pd.read_csv(path_gt, header= None, names=['frame', 'track_id', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)
det = pd.read_csv(path_det, header= None, names=['frame', 'unk1', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)

gt.head()


,frame,track_id,x_cord,y_cord,width,height,acc,unk2,unk3,ball_check
0,1,1,909,483,47,131,1,-1,-1,-1
1,2,1,909,483,47,131,1,-1,-1,-1
2,3,1,909,483,47,130,1,-1,-1,-1
3,4,1,910,483,47,130,1,-1,-1,-1
4,5,1,910,484,47,131,1,-1,-1,-1


# ciao

In [ ]:
gt['area']= gt.width * gt.height
gt['dim_sum']= np.abs(gt.width + gt.height)
gt_track_id = gt.groupby('track_id').mean()
gt_count_frame = gt.groupby('track_id').count()
gt_count_frame.sort_values(['acc'], ascending=False, inplace=True)

dim_ok = gt_track_id[gt_track_id.dim_sum < 50]

#print(gt_track_id[gt_track_id.area == gt_track_id.area.min()]) # entità con l'area  inore
#print(dim_ok) # entità con la somma delle fuperfici minore di 50

# aggiungere un check di sicurezza ?

max_frame =gt_count_frame[gt_count_frame['acc'] == gt_count_frame.max().acc] # indice del oggetto presente in più frame

flag = False

while flag :
    # check se il valore con più frame ha ua somma dell dimensioni che va bene
    print(flag)

res = dim_ok



print('res\n', res)

list_frames=gt[gt.track_id == res.index[0]].frame

if len(res.index) > 1:
    for i in range(1, len(res.index)) :
        list_frames = pd.concat([list_frames, gt[gt.track_id == res.index[i]].frame], ignore_index=True)# risalgo ai frame dove il track id è la palla

print('\nlist_frames\n',list_frames)



for frame in list_frames:
    actual_frame = gt[(gt.track_id.isin(res.index)) & (gt.frame == frame)] # la riga effettiva delle informazioni sul frame e l'indice
    gt.loc[(gt.width == actual_frame.width.values[0]) & (gt.frame == frame) &(gt.x_cord == actual_frame.x_cord.values[0]), 'ball_check'] = 1
    #print(np.sum([(gt.track_id == res.index[0]) & (gt.frame == frame)]))
    #print((det.frame == frame)&(det.width == actual_frame.width.values[0]))
    det.loc[(det.frame == frame)&(det.width == actual_frame.width.values[0]) &(det.x_cord == actual_frame.x_cord.values[0]), 'ball_check'] = 1
    # &(det.x_cord == actual_frame.x_cordvalues[0])


det[det.ball_check == 1] #in ogni frame quale indice è descrittto come palla

res
               frame       x_cord      y_cord      width     height  acc  unk2  \
track_id                                                                        
11        149.01487  1046.743494  516.297398  23.959108  23.252788  1.0  -1.0   
26        480.00000  1097.000000  738.000000  20.000000  22.000000  1.0  -1.0   
27        598.00000  1279.400000  704.157447  20.000000  22.000000  1.0  -1.0   
29        733.00000  1156.114286  573.057143  18.371429  19.771429  1.0  -1.0   

          unk3  ball_check        area    dim_sum  
track_id                                           
11        -1.0         1.0  573.817844  47.211896  
26        -1.0        -1.0  440.000000  42.000000  
27        -1.0        -1.0  440.000000  42.000000  
29        -1.0        -1.0  367.800000  38.142857  

list_frames
 0        1
1        2
2        3
3        4
4        5
      ... 
535    746
536    747
537    748
538    749
539    750
Name: frame, Length: 540, dtype: int64


,frame,unk1,x_cord,y_cord,width,height,acc,unk2,unk3,ball_check
10,1,-1,1255,608,22,24,1,-1,-1,1
22,2,-1,1255,608,22,24,1,-1,-1,1
34,3,-1,1255,608,22,24,1,-1,-1,1
46,4,-1,1255,608,22,24,1,-1,-1,1
58,5,-1,1255,609,22,24,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...
10696,746,-1,1071,484,14,15,1,-1,-1,1
10715,747,-1,1070,483,14,15,1,-1,-1,1
10734,748,-1,1065,477,14,14,1,-1,-1,1
10753,749,-1,1064,477,14,14,1,-1,-1,1


In [ ]:
gt_count_frame.index.intersection(dim_ok.index)
#gt_count_frame.index

Int64Index([11, 27, 29, 26], dtype='int64', name='track_id')

In [ ]:
gt_count_frame


,frame,x_cord,y_cord,width,height,acc,unk2,unk3,ball_check,area,dim_sum
track_id,,,,,,,,,,,
8,585,585,585,585,585,585,585,585,585,585,585
3,582,582,582,582,582,582,582,582,582,582,582
6,534,534,534,534,534,534,534,534,534,534,534
2,532,532,532,532,532,532,532,532,532,532,532
4,522,522,522,522,522,522,522,522,522,522,522
7,519,519,519,519,519,519,519,519,519,519,519
9,512,512,512,512,512,512,512,512,512,512,512
5,509,509,509,509,509,509,509,509,509,509,509
1,506,506,506,506,506,506,506,506,506,506,506


# ciao

In [ ]:
gt[gt.ball_check == 1] # il tracking della palla su tutti i frame della cartella


,frame,track_id,x_cord,y_cord,width,height,acc,unk2,unk3,ball_check,area,dim_sum
5214,1,11,1255,608,22,24,1,-1,-1,1,528,46
5215,2,11,1255,608,22,24,1,-1,-1,1,528,46
5216,3,11,1255,608,22,24,1,-1,-1,1,528,46
5217,4,11,1255,608,22,24,1,-1,-1,1,528,46
5218,5,11,1255,609,22,24,1,-1,-1,1,528,46
...,...,...,...,...,...,...,...,...,...,...,...,...
10770,746,29,1071,484,14,15,1,-1,-1,1,210,29
10771,747,29,1070,483,14,15,1,-1,-1,1,210,29
10772,748,29,1065,477,14,14,1,-1,-1,1,196,28
10773,749,29,1064,477,14,14,1,-1,-1,1,196,28


In [ ]:
gt = gt.drop(['area', 'dim_sum', 'unk2', 'unk3'], axis=1)
det = det.drop(['unk2', 'unk3'], axis=1)

gt.to_csv(save_gt+'/gt.txt', index= False)
det.to_csv(save_det+'/det.txt', index= False)

In [ ]:
def def_path(folder):
    path_to_txt_files ='/media/alb/Elements/DATASET/SoccerNet/tracking/test/' + folder
    path_det = path_to_txt_files + '/det/det.txt'
    path_gt = path_to_txt_files + '/gt/gt.txt'

    gt = pd.read_csv(path_gt, header= None, names=['frame', 'track_id', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)
    det = pd.read_csv(path_det, header= None, names=['frame', 'unk1', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)

    return det, gt

def def_path_to_save(folder):
    path_to_save ='/media/alb/Elements/DATASET/SoccerNet/tracking/proj_DL/proj_test'
    save_gt = path_to_save + '/proj_gt'
    save_det = path_to_save + '/proj_det'
    save_img1 = path_to_save + '/proj_img1'

        

In [ ]:
def mod_dataset(det: pd.DataFrame , gt: pd.DataFrame, i=0):

    gt['area']= gt.width * gt.height
    gt['dim_sum']= np.abs(gt.width + gt.height)
    gt_track_id = gt.groupby('track_id').mean()
    gt_count_frame = gt.groupby('track_id').count()

    # aggiungere un check di sicurezza ?

    res=gt_count_frame[gt_count_frame['acc'] == gt_count_frame.max().acc] # indice della palla

    list_frames = gt[gt.track_id == res.index[0]].frame # risalgo ai frame dove il track id è la palla

    for frame in list_frames:
        actual_frame = gt[(gt.track_id == res.index[0]) & (gt.frame == frame)] # la riga effettiva delle informazioni sul frame e l'indice
        gt.loc[(gt.width == actual_frame.width.values[0]) & (gt.frame == frame) &(gt.x_cord == actual_frame.x_cord.values[0]), 'ball_check'] = 1
        det.loc[(det.frame == frame)&(det.width == actual_frame.width.values[0]) &(det.x_cord == actual_frame.x_cord.values[0]), 'ball_check'] = 1

    gt.frame.map(lambda x: x + i*750)
    det.frame.map(lambda x: x + i*750)

    gt = gt.drop(['area', 'dim_sum', 'unk2', 'unk3'], axis=1)
    det = det.drop(['unk2', 'unk3'], axis=1)
    
    return det, gt


In [ ]:
i = 0
final_det, final_gt = pd.DataFrame(), pd.DataFrame()

for folder in path_folder:

    det, gt = def_path(folder)
    new_det, new_gt = mod_dataset(det, gt)

    choosen_img = rnd.sample(range(750),200)

    v = [0 for _ in range(750)]

    for num in rnd.sample(range(750),200):
        i+=1
        path_old_img = '...'
        save_new_img ='...'
        final_det = final_det.append(new_det[new_det.frame == num])
        final_gt = final_gt.append(new_gt[new_gt.frame == num])

        

In [ ]:
k = [rnd.randint(0,1) for i in range(10)]
k

[0, 1, 1, 1, 1, 1, 0, 0, 0, 1]

In [ ]:
choosen_img = rnd.sample(range(750),200)

v = [False for _ in range(750)]

for i in choosen_img: v[i] = True

[det.frame.values in choosen_img]


In [2]:
path_cartella_cartelle = ('/media/alb/Elements/DATASET/SoccerNet/tracking/train')
cartella_cartelle = os.listdir('/media/alb/Elements/DATASET/SoccerNet/tracking/train')
cartella_cartelle.sort()

for numero_cartella in range(2): # vado a cercare la cartella delle immagini + gt + det
    print(numero_cartella)

    path_cartella_immagini = path_cartella_cartelle + '/' + cartella_cartelle[numero_cartella] + '/img1/' # defnisco il path per la cartella delle immagini
    path_cartella_gt = path_cartella_cartelle + '/' + cartella_cartelle[numero_cartella] + '/gt/'
    path_cartella_det = path_cartella_cartelle + '/' + cartella_cartelle[numero_cartella] + '/det/'

    path_gt = path_cartella_gt + 'gt.txt'
    path_det = path_cartella_det + 'det.txt'

    gt = pd.read_csv(path_gt, header= None, names=['frame', 'track_id', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)
    det = pd.read_csv(path_det, header= None, names=['frame', 'unk1', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)

    new_det, new_gt = mod_dataset(det, gt, i = numero_cartella +1)

    path_to_save ='/media/alb/Elements/DATASET/SoccerNet/tracking/proj_DL/proj_test'
    save_gt = path_to_save + '/proj_gt'
    save_det = path_to_save + '/proj_det'
    save_img1 = path_to_save + '/proj_img1/'
    
    if numero_cartella ==0 : #crea unnuovo file

        new_gt.to_csv(save_gt+'/gt.txt')
        new_det.to_csv(save_det+'/det.txt')


    if numero_cartella > 0 : # aggiungi testo a file esistente

        file = open(save_gt+'/gt.txt', 'a')
        gt_as_string = new_gt.to_string(header=False)
        file.write(gt_as_string)
        file.close()

        file = open(save_det+'/det.txt', 'a')
        det_as_string = new_det.to_string(header=False)
        file.write(det_as_string)
        file.close()

    cartella_immagini = os.listdir(path_cartella_immagini)
    cartella_immagini.sort()

    for immagine in cartella_immagini: # le singloe immagini nella cartella img1
        nome_imm = immagine[-7:]
        print(save_img1 + str( 750 * (numero_cartella + 1) + int(nome_imm[:3])))
        path_immagine = path_cartella_immagini + immagine #  path finale per trovre queste benedette immagini
        img = os.open(save_img1 + str( 750 * (numero_cartella + 1) + int(nome_imm[:3])) + '.jpg', 'wb')
        img.write(path_immagine)
        img.close()
        #img = cv2.imread(path_immagine) # apro l'mmagine
        #cv2.imsave(save_img1 + str( 750 * (numero_cartella +1) + int(nome_imm[:3])), img) # la salvo con un altro nome in un altra posizione

# problema: la cartella potrebbe essere molto molto pesante

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: '/media/alb/Elements/DATASET/SoccerNet/tracking/train'

# MIO

In [3]:

from pathlib import Path  

#path_cartella_cartelle = (r'C:\Users\david\Desktop\CORSI ORAAA\Deep Learing\PROGETTO DEEP\FILE_MANADARE')
#cartella_cartelle = os.listdir(r'C:\Users\david\Desktop\CORSI ORAAA\Deep Learing\PROGETTO DEEP\FILE_MANADARE')

path_cartella_cartelle = (r'C:\Users\david\Desktop\CORSI ORAAA\Deep Learing\PROGETTO DEEP\CARTELLE_TOT')
cartella_cartelle = os.listdir(r'C:\Users\david\Desktop\CORSI ORAAA\Deep Learing\PROGETTO DEEP\CARTELLE_TOT')


filepath = Path(r'C:\Users\david\Desktop\CORSI ORAAA\Deep Learing\PROGETTO DEEP\CARTELLE_SPLIT\111123')  ###where to save

##path_cartella_cartelle = ('/media/alb/Elements/DATASET/SoccerNet/tracking/train')



In [9]:
cartella_cartelle.sort()
print(cartella_cartelle[2])

SNMOT-116


In [96]:
#path_cartella_cartelle = ('/media/alb/Elements/DATASET/SoccerNet/tracking/train')
#cartella_cartelle = os.listdir('/media/alb/Elements/DATASET/SoccerNet/tracking/train')
cartella_cartelle.sort()


#,frame,unk1,x_cord,y_cord,width,height,acc,ball_check


for numero_cartella in range(3): # vado a cercare la cartella delle immagini + gt + det
    print("numero cartella",numero_cartella)

    path_cartella_immagini = path_cartella_cartelle + '/' + cartella_cartelle[numero_cartella] + '/img1/' # defnisco il path per la cartella delle immagini
    path_cartella_gt = path_cartella_cartelle + '/' + cartella_cartelle[numero_cartella] + '/gt/'
    path_cartella_det = path_cartella_cartelle + '/' + cartella_cartelle[numero_cartella] + '/det/'

    path_gt = path_cartella_gt + 'gt.txt'
    path_det = path_cartella_det + 'det.txt'

    #gt = pd.read_csv(path_gt, header= None, names=['frame', 'track_id', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)
    #det = pd.read_csv(path_det, header= None, names=['frame', 'unk1', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)
    
    #### ORA USO
    #gt = pd.read_csv(path_gt, header= 0, names=['frame', 'track_id', 'x_cord', 'y_cord', 'width', 'height', 'acc',  'ball_check'], index_col=False)
    det = pd.read_csv(path_det, header= 0, names=['frame', 'unk1', 'x_cord', 'y_cord', 'width', 'height', 'acc',  'ball_check' , "team_label"], index_col=None)  
    df=det
    #print(det)
    
    
    
    for i in range(750):         #### se non sempre 750 lavora su questo
        df_1 = df[df["frame"] == i]################cambia nome colonna
        
        #filepath.parent.mkdir(parents=True, exist_ok=True)  
        
        
        #C:\Users\david\Desktop\CORSI ORAAA\Deep Learing\PROGETTO DEEP
        df_1.to_csv(  r'C:\Users\david\Desktop\CORSI ORAAA\Deep Learing\PROGETTO DEEP\CARTELLE_SPLIT\SNMOT-' + str(numero_cartella) +"_" +str(i)+'.txt' , index=False ,sep=' ', mode='a')   
        #df_1.to_csv(  'path_to_save_file_' + str(numero_cartella) +"_" +str(i)+'.csv' , index=False)   
        ##df.iloc[ i   ,:].to_csv('path_to_save_file_'+str(i*10**5)+"_"+str(i*10**5)+'.csv')  reference
        print("Saving file with row: ",i)
        
        
        
    print(2)
        
        
        




numero cartella 0
Saving file with row:  0
Saving file with row:  1
Saving file with row:  2
Saving file with row:  3
Saving file with row:  4
Saving file with row:  5
Saving file with row:  6
Saving file with row:  7
Saving file with row:  8
Saving file with row:  9
Saving file with row:  10
Saving file with row:  11
Saving file with row:  12
Saving file with row:  13
Saving file with row:  14
Saving file with row:  15
Saving file with row:  16
Saving file with row:  17
Saving file with row:  18
Saving file with row:  19
Saving file with row:  20
Saving file with row:  21
Saving file with row:  22
Saving file with row:  23
Saving file with row:  24
Saving file with row:  25
Saving file with row:  26
Saving file with row:  27
Saving file with row:  28
Saving file with row:  29
Saving file with row:  30
Saving file with row:  31
Saving file with row:  32
Saving file with row:  33
Saving file with row:  34
Saving file with row:  35
Saving file with row:  36
Saving file with row:  37
Savi

In [14]:
############ MIO
                     ## QUI /
for i in range(750):         #### se non sempre 750 lavora su questo
    df_1 = df[df["group_check"] == i]################cambia nome colonna
    df_1.to_csv('path_to_save_file_'+str(i*)+".csv')
   ##df.iloc[ i   ,:].to_csv('path_to_save_file_'+str(i*10**5)+"_"+str(i*10**5)+'.csv')  reference
print("Saving file with row: ",i)

SyntaxError: invalid syntax (3589179311.py, line 5)

In [ ]:
int('009')

9

In [ ]:
import os

In [ ]:
cartella_cartelle = os.listdir('/media/alb/Elements/DATASET/SoccerNet/tracking/train')
cartella_cartelle.sort()
cartella_cartelle 

['SNMOT-060',
 'SNMOT-061',
 'SNMOT-062',
 'SNMOT-063',
 'SNMOT-064',
 'SNMOT-065',
 'SNMOT-066',
 'SNMOT-067',
 'SNMOT-068',
 'SNMOT-069',
 'SNMOT-070',
 'SNMOT-071',
 'SNMOT-072',
 'SNMOT-073',
 'SNMOT-074',
 'SNMOT-075',
 'SNMOT-076',
 'SNMOT-077',
 'SNMOT-097',
 'SNMOT-098',
 'SNMOT-099',
 'SNMOT-100',
 'SNMOT-101',
 'SNMOT-102',
 'SNMOT-103',
 'SNMOT-104',
 'SNMOT-105',
 'SNMOT-106',
 'SNMOT-107',
 'SNMOT-108',
 'SNMOT-109',
 'SNMOT-110',
 'SNMOT-111',
 'SNMOT-112',
 'SNMOT-113',
 'SNMOT-114',
 'SNMOT-115',
 'SNMOT-167',
 'SNMOT-168',
 'SNMOT-169',
 'SNMOT-170',
 'SNMOT-187',
 'SNMOT-188',
 'SNMOT-189',
 'SNMOT-190',
 'SNMOT-191',
 'SNMOT-192',
 'SNMOT-193',
 'SNMOT-194',
 'SNMOT-195',
 'SNMOT-196',
 'SNMOT-197',
 'SNMOT-198',
 'SNMOT-199',
 'SNMOT-200']

In [ ]:
cartella = os.listdir('/media/alb/Elements/DATASET/SoccerNet/tracking/train/SNMOT-061/img1')
cartella.sort()
for nome_immagine in cartella:
    print(nome_immagine[-7:])

001.jpg
002.jpg
003.jpg
004.jpg
005.jpg
006.jpg
007.jpg
008.jpg
009.jpg
010.jpg
011.jpg
012.jpg
013.jpg
014.jpg
015.jpg
016.jpg
017.jpg
018.jpg
019.jpg
020.jpg
021.jpg
022.jpg
023.jpg
024.jpg
025.jpg
026.jpg
027.jpg
028.jpg
029.jpg
030.jpg
031.jpg
032.jpg
033.jpg
034.jpg
035.jpg
036.jpg
037.jpg
038.jpg
039.jpg
040.jpg
041.jpg
042.jpg
043.jpg
044.jpg
045.jpg
046.jpg
047.jpg
048.jpg
049.jpg
050.jpg
051.jpg
052.jpg
053.jpg
054.jpg
055.jpg
056.jpg
057.jpg
058.jpg
059.jpg
060.jpg
061.jpg
062.jpg
063.jpg
064.jpg
065.jpg
066.jpg
067.jpg
068.jpg
069.jpg
070.jpg
071.jpg
072.jpg
073.jpg
074.jpg
075.jpg
076.jpg
077.jpg
078.jpg
079.jpg
080.jpg
081.jpg
082.jpg
083.jpg
084.jpg
085.jpg
086.jpg
087.jpg
088.jpg
089.jpg
090.jpg
091.jpg
092.jpg
093.jpg
094.jpg
095.jpg
096.jpg
097.jpg
098.jpg
099.jpg
100.jpg
101.jpg
102.jpg
103.jpg
104.jpg
105.jpg
106.jpg
107.jpg
108.jpg
109.jpg
110.jpg
111.jpg
112.jpg
113.jpg
114.jpg
115.jpg
116.jpg
117.jpg
118.jpg
119.jpg
120.jpg
121.jpg
122.jpg
123.jpg
124.jpg
125.jpg
